In [56]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
import datetime
from datetime import date
from random import randint
import numpy as np
from shapely.geometry import Point, LineString, Polygon
import pysal as ps

from bokeh.models import *
from bokeh.models.callbacks import *
from bokeh.plotting import *
from bokeh.io import *
from bokeh.tile_providers import *
from bokeh.palettes import *
from bokeh.transform import *
from bokeh.layouts import *
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

import warnings

import os

output_notebook()

warnings.filterwarnings("ignore")
# pd.set_option("max_columns", 30)

output_notebook()

## DATA ACLED ##

# path to ACLED conflict data
fp_acled = r'UKR_ACLED/CSV_2022-01-01-2022-11-08-Belarus-Moldova-Russia-Ukraine.csv'
# read ACLED conflict data into dataframe
df_acled = pd.read_csv(fp_acled)

# create a geodataframe out of 'conflict_data'
geo_acled = gpd.GeoDataFrame(df_acled, geometry=gpd.points_from_xy(df_acled.longitude, df_acled.latitude), crs=CRS.from_epsg(4326).to_wkt())

# Convert event date to pd.datetime format
geo_acled['event_date'] = pd.to_datetime(geo_acled['event_date'], errors='coerce')

# transform to Mercator CRS needed by Bokeh
geo_acled = geo_acled.to_crs(epsg=3857)

# Creating lat and lon in Mercator
geo_acled['mercator_lon'] = geo_acled['geometry'].x
geo_acled['mercator_lat'] = geo_acled['geometry'].y

# creating bins
cut_bins = [0, 1, 20, 50, 100, 250, 500]
geo_acled['bin_fatalities'] = pd.cut(geo_acled['fatalities'] , bins=cut_bins, labels=[3,5,8,15,25,40], include_lowest=True)

## remove unwanted layers e.g. certain event_types or certain actors
# list of all event types:
#    'Battles',
#    'Explosions/Remote violence',
#    'Protests',
#    'Riots',
#    'Strategic developments',
#    'Violence against civilians'

unwanted_layers = ['Protests','Riots','Strategic developments']
layers_needed = [i for i in geo_acled['event_type'].unique().tolist() if i not in unwanted_layers]
# in: layers_needed
# out: ['Explosions/Remote violence', 'Battles', 'Violence against civilians']

# limit dataframe to records where event_type are not in the unwanted list
geo_acled_limited = geo_acled.loc[geo_acled['event_type'].isin(layers_needed)]
geo_acled_limited = geo_acled_limited.reset_index(drop=True)

# CDS data Container for Bokeh
geo_acled_limited = geo_acled_limited.drop('geometry', axis=1)

source_acled = ColumnDataSource(geo_acled_limited)

# Create list of all actors in actor1 column
options_actor1 = geo_acled_limited['actor1'].dropna().drop_duplicates().tolist()
options_actor1.sort()

def modify_doc(doc):
    
    def make_plot(src):
        # Initialize the plot (p) and give it a title
        tile_provider = get_provider(STAMEN_TONER)

        #The range for the map extents is derived from the lat/lon fields. This way the map is automatically centered on the plot elements.
        scale = 6000
        x=geo_acled_limited['mercator_lon']
        y=geo_acled_limited['mercator_lat']

        x_min=int(x.mean() - (scale*350))
        x_max=int(x.mean() + (scale*350))
        y_min=int(y.mean() - (scale*350))
        y_max=int(y.mean() + (scale*350))

        p = figure(plot_width=1000, plot_height=700,
                           x_range=(x_min, x_max),
                           y_range=(y_min, y_max),
                           x_axis_type="mercator", y_axis_type="mercator",
                           match_aspect=True,
                           tools='box_zoom,wheel_zoom,pan,reset,save',
                           title="2022 Ukraine Conflict Data\nSource: www.acleddata.com​.")

        p.circle(x='mercator_lon', y='mercator_lat',fill_color='red',line_color='red'
                        ,hover_color='yellow',size='bin_fatalities',fill_alpha=0.4,source=geo_acled_limited)

        p.grid.visible=True

        m=p.add_tile(tile_provider)
        m.level='underlay'

        return p

    def get_dataset(actors1_list):
        subset = geo_acled_limited[geo_acled_limited['actor1'].isin(actors1_list)]
        new_src = ColumnDataSource(subset)

    # Update function takes three default parameters
    def update(attr, old, new):
        # Get the list of actors for the map
        actors1_list = [options_actor1[i] for i in actor1_select.active]
        # Make a new dataset based on the selected actors and the get_dataset function defined earlier
        new_src = get_dataset(actors1_list)

        # Convert dataframe to column data source
        new_src = ColumnDataSource(new_src)

        src.data.update(new_src.data)    

    # ### CHECKBOX WITH ACTORS1 FROM ACLED
    # # Create list of all actors in actor1 column
    # options_actor1 = geo_df['actor1'].dropna().drop_duplicates().tolist()
    # options_actor1.sort()

    # Create checkboxgroup widget to allow selection of ACLED actors
    actor1_select = CheckboxGroup(labels=options_actor1,active = [0])
    # Link a change in selected buttons to the update function
    actor1_select.on_change('active', update)

    src = get_dataset([options_actor1[i] for i in actor1_select.active])

    controls = WidgetBox(actor1_select)

    p = make_plot(src)

    layout = row(controls, p)
    doc.add_root(layout)

# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

Loading BokehJS ...

Loading BokehJS ...